In [ ]:
import pandas as pd

def carregar_e_limpar(caminho_arquivo):
    df = pd.read_csv(caminho_arquivo, sep=',')
    return df

def adicionar_variaveis_temporais(df):
    if 'data_iniSE' in df.columns:
        df['data_iniSE'] = pd.to_datetime(df['data_iniSE'])
        df['epidemiological_year'] = df['data_iniSE'].dt.year
        df['epidemiological_week'] = df['data_iniSE'].dt.isocalendar().week
    else:
        print("A coluna não encontrada no DataFrame.")

    if 'casos' in df.columns:
        df['casos_lag_1'] = df['casos'].shift(1)
        df['casos_lag_2'] = df['casos'].shift(2)
        df['casos_lag_3'] = df['casos'].shift(3)
        df['casos_media_3'] = df[['casos_lag_1', 'casos_lag_2', 'casos_lag_3']].mean(axis=1)
    else:
        print("A coluna não encontrada no DataFrame.")

    return df

def remover_outliers(df, features):
    df_sem_outliers = df.copy()
    for feature in features:
        if feature in df.columns:
            Q1 = df[feature].quantile(0.25)
            Q3 = df[feature].quantile(0.75)
            IQR = Q3 - Q1
            limite_inferior = Q1 - 1.5 * IQR
            limite_superior = Q3 + 1.5 * IQR
            df_sem_outliers = df_sem_outliers[
                (df_sem_outliers[feature] >= limite_inferior) & (df_sem_outliers[feature] <= limite_superior)
            ]
        else:
            print(f"Feature '{feature}' não encontrada no DataFrame.")

    return df_sem_outliers